# Setup use case Wine 

### Set up the auto completion

In [ ]:
#install auto-completion 
#!pip install jupyter_contrib_nbextensions
#!jupyter contrib nbextension install --user
#from jedi import settings
#settings.case_insensitive_completion = True

### Set up the environment 
- Python 3
- virtualenv
- Jupyter notebook running in a virtual environment with Python 3
- Docker
- Git
- Google Cloud SDK (gcloud)

### Enable  the APIs if they are not enabled 

In [ ]:
!gcloud services enable compute.googleapis.com         \
                       containerregistry.googleapis.com  \
                       aiplatform.googleapis.com  \
                       cloudbuild.googleapis.com \
                       cloudfunctions.googleapis.com

ERROR: (gcloud.services.enable) User [829340160841-compute@developer.gserviceaccount.com] does not have permission to access projects instance [spmena-sandbox-in-apac-dv] (or it may not exist): Permission denied to enable service [compute.googleapis.com]
Permission denied to enable service [containerregistry.googleapis.com]
Permission denied to enable service [aiplatform.googleapis.com]
Permission denied to enable service [cloudbuild.googleapis.com]
Permission denied to enable service [cloudfunctions.googleapis.com]
Help Token: AXyI5c8bzCKH7ahC8lZr8672i3LC1Of04gMg82HAiFFdpNrFWEsi-tlXi7JQeF2SXhledBr7qzU5Y0VH_nTDpH87Ki_Z-MmSkOj1b5lZY9sAxagb
- '@type': type.googleapis.com/google.rpc.PreconditionFailure
  violations:
  - subject: ?error_code=110002&service=serviceusage.googleapis.com&service=serviceusage.googleapis.com&service=serviceusage.googleapis.com&service=serviceusage.googleapis.com&service=serviceusage.googleapis.com&permission=serviceusage.services.enable&permission=serviceusage.s

In [1]:
!gcloud services enable cloudresourcemanager.googleapis.com  --project $PROJECT_ID

ERROR: (gcloud.services.enable) argument --project: expected one argument
Usage: gcloud services enable [SERVICE ...] [optional flags]
  optional flags may be  --async | --help

For detailed information on this command and its flags, run:
  gcloud services enable --help


In [2]:
!PROJECT_ID="spmena-sandbox-in-apac-dv"

## Uncomment all needed cells if you need to install missing packages 

In [3]:
# Install ai platform and kfp
!pip3 install  google-cloud-aiplatform==1.3.0 --upgrade --user
!pip3 install  kfp --upgrade --user

  Using cached google_cloud_aiplatform-1.3.0-py2.py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.21.0
    Uninstalling google-cloud-aiplatform-1.21.0:
      Successfully uninstalled google-cloud-aiplatform-1.21.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-pipeline-components 1.0.33 requires google-cloud-aiplatform<2,>=1.11.0, but you have google-cloud-aiplatform 1.3.0 which is incompatible.


In [4]:
!pip install google_cloud_pipeline_components --user

  Using cached google_cloud_aiplatform-1.21.0-py2.py3-none-any.whl (2.3 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.3.0
    Uninstalling google-cloud-aiplatform-1.3.0:
      Successfully uninstalled google-cloud-aiplatform-1.3.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
#!gcloud auth login if needed

In [5]:
# Restart the kernel
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)


#### Set up the global variables 

In [1]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-central1"

# Get projet name
shell_output=!gcloud config get-value project 2> /dev/null
PROJECT_ID=shell_output[0]

# Set bucket name
BUCKET_NAME="ae2-in-gcs-dv-assortment"

# Create bucket
#PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root_wine/"
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/pipeline_root_wine/"

USER_FLAG = "--user"


env: PATH=/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/jupyter/.local/bin


## Import Libraries
 * Artifact,
 * Dataset,
 * Input,
 * Model,
 * Output,
 * Metrics,
 * ClassificationMetrics
 * InputPath
 * OutputPath

In [2]:
from typing import NamedTuple
import typing
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)

from kfp.v2 import compiler
from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components import aiplatform as gcc_aip



### Create pipeline

We create 4 components:  
- Load data   
- Train a  model
- Evaluate the model 
- Deploy the model

The components have dependencies on `pandas`, `sklearn`.

In [3]:
# List columns
import pandas as pd
df_wine = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", delimiter=";")
df_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


Available columns: 
- volatile acidity :   Volatile acidity is the gaseous acids present in wine.
- fixed acidity :   Primary fixed acids found in wine are tartaric, succinic, citric, and malic
- residual sugar :   Amount of sugar left after fermentation.
- citric acid :    It is weak organic acid, found in citrus fruits naturally.
- chlorides :   Amount of salt present in wine.
- free sulfur dioxide :   So2 is used for prevention of wine by oxidation and microbial spoilage.
- total sulfur dioxide 
- pH :   In wine pH is used for checking acidity
- density 
- sulphates :    Added sulfites preserve freshness and protect wine from oxidation, and bacteria.
- alcohol :   Percent of alcohol present in wine.



### Read the wine quality dataset 

In [5]:
@component(
    packages_to_install=["pandas", "pyarrow", "scikit-learn==1.0.0"],
    base_image="python:3.9",
    output_component_file="get_wine_data.yaml"
)

def get_wine_data(
    url: str,
    dataset_train: Output[Dataset],
    dataset_test: Output[Dataset]
):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split as tts
    
    df_wine = pd.read_csv(url, delimiter=";")
    df_wine['best_quality'] = [ 1 if x>=7 else 0 for x in df_wine.quality] 
    df_wine['target'] = df_wine.best_quality
    df_wine = df_wine.drop(['quality', 'total sulfur dioxide', 'best_quality'], axis=1)
   
   
    train, test = tts(df_wine, test_size=0.3)
    train.to_csv(dataset_train.path + ".csv" , index=False, encoding='utf-8-sig')
    test.to_csv(dataset_test.path + ".csv" , index=False, encoding='utf-8-sig')
    


#### Train the wine quality model


In [6]:
@component(
    packages_to_install = ["pandas", "pyarrow", "scikit-learn==1.0.0"
    ], base_image="python:3.9",
)
def train_winequality(
    dataset:  Input[Dataset],
    model: Output[Model], 
):
    
    from sklearn.ensemble import RandomForestClassifier
    import pandas as pd
    import pickle

    data = pd.read_csv(dataset.path+".csv")
    model_rf = RandomForestClassifier(n_estimators=10)
    model_rf.fit(
        data.drop(columns=["target"]),
        data.target,
    )
    model.metadata["framework"] = "RF"
    file_name = model.path + f".pkl"
    with open(file_name, 'wb') as file:  
        pickle.dump(model_rf, file)
        

#### Evaluate the model 

In [7]:
@component(
    packages_to_install = ["pandas", "pyarrow", "scikit-learn==1.0.0"], base_image="python:3.9",
)
def winequality_evaluation(
    test_set:  Input[Dataset],
    rf_winequality_model: Input[Model],
    thresholds_dict_str: str,
    metrics: Output[ClassificationMetrics],
    kpi: Output[Metrics]
) -> NamedTuple("output", [("deploy", str)]):

    from sklearn.ensemble import RandomForestClassifier
    import pandas as pd
    import logging 
    import pickle
    from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score
    import json
    import typing

    
    def threshold_check(val1, val2):
        cond = "false"
        if val1 >= val2 :
            cond = "true"
        return cond

    data = pd.read_csv(test_set.path+".csv")
    model = RandomForestClassifier()
    file_name = rf_winequality_model.path + ".pkl"
    with open(file_name, 'rb') as file:  
        model = pickle.load(file)
    
    y_test = data.drop(columns=["target"])
    y_target=data.target
    y_pred = model.predict(y_test)
    

    y_scores =  model.predict_proba(data.drop(columns=["target"]))[:, 1]
    fpr, tpr, thresholds = roc_curve(
         y_true=data.target.to_numpy(), y_score=y_scores, pos_label=True
    )
    metrics.log_roc_curve(fpr.tolist(), tpr.tolist(), thresholds.tolist())  
    
    metrics.log_confusion_matrix(
       ["False", "True"],
       confusion_matrix(
           data.target, y_pred
       ).tolist(), 
    )
    
    accuracy = accuracy_score(data.target, y_pred.round())
    thresholds_dict = json.loads(thresholds_dict_str)
    rf_winequality_model.metadata["accuracy"] = float(accuracy)
    kpi.log_metric("accuracy", float(accuracy))
    deploy = threshold_check(float(accuracy), int(thresholds_dict['roc']))
    return (deploy,)
    

    

### Deploy model 

In [8]:
@component(
    packages_to_install=["google-cloud-aiplatform", "pandas", "pyarrow", "scikit-learn==1.0.0", "kfp"],
    #base_image="europe-west1-docker.pkg.dev/PROJECT_ID/wine-train/scikit-learn-winequality:latest",
    base_image="python:3.9",
    output_component_file="model_winequality_coponent.yml"
)
def deploy_winequality(
    model: Input[Model],
    project: str,
    region: str,
    serving_container_image_uri : str, 
    vertex_endpoint: Output[Artifact],
    vertex_model: Output[Model]
):
    from google.cloud import aiplatform
    aiplatform.init(project=project, location=region)

    DISPLAY_NAME  = "winequality-new"
    MODEL_NAME = "winequality-rf-new"
    ENDPOINT_NAME = "winequality_endpoint_new"
    
    def create_endpoint():
        endpoints = aiplatform.Endpoint.list(
        filter='display_name="{}"'.format(ENDPOINT_NAME),
        order_by='create_time desc',
        project=project, 
        location=region,
        )
        if len(endpoints) > 0:
            endpoint = endpoints[0]  # most recently created
        else:
            endpoint = aiplatform.Endpoint.create(
            display_name=ENDPOINT_NAME, project=project, location=region
        )
    endpoint = create_endpoint()   
    
    
    #Import a model programmatically
    model_upload = aiplatform.Model.upload(
        display_name = DISPLAY_NAME, 
        artifact_uri = model.uri.replace("model", ""),
        serving_container_image_uri =  serving_container_image_uri,
        serving_container_health_route=f"/v1/models/{MODEL_NAME}",
        serving_container_predict_route=f"/v1/models/{MODEL_NAME}:predict",
        serving_container_environment_variables={
        "MODEL_NAME": MODEL_NAME,
    },       
    )
    model_deploy = model_upload.deploy(
        machine_type="n1-standard-4", 
        endpoint=endpoint,
        traffic_split={"0": 100},
        deployed_model_display_name=DISPLAY_NAME,
    )

    # Save data to the output params
    vertex_model.uri = model_deploy.resource_name

In [10]:
from datetime import datetime

TIMESTAMP =datetime.now().strftime("%Y%m%d%H%M%S")
DISPLAY_NAME = 'pipeline-winequality-job-new{}'.format(TIMESTAMP)

In [11]:
!$TIMESTAMP

/bin/bash: 20230118201821: command not found


#### Create the Pipeline

Once you have created all the needed components define the pipeline and then compile it into a `.json` file.

In [12]:
@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline. Use to determine the pipeline Context.
    name="pipeline-winequality-new",
    
)
def pipeline(
    url: str = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv",
    project: str = PROJECT_ID,
    region: str = REGION, 
    display_name: str = DISPLAY_NAME,
    api_endpoint: str = REGION+"-aiplatform.googleapis.com",
    thresholds_dict_str: str = '{"roc":0.8}',
    serving_container_image_uri: str = "europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest"
    ):
    
    data_op = get_wine_data(url)
    train_model_op = train_winequality(data_op.outputs["dataset_train"])
    model_evaluation_op = winequality_evaluation(
        test_set=data_op.outputs["dataset_test"],
        rf_winequality_model=train_model_op.outputs["model"],
        thresholds_dict_str = thresholds_dict_str, # I deploy the model anly if the model performance is above the threshold
    )
    
    with dsl.Condition(
        model_evaluation_op.outputs["deploy"]=="true",
        name="deploy-winequality-new",
    ):
           
        deploy_model_op = deploy_winequality(
        model=train_model_op.outputs['model'],
        project=project,
        region=region, 
        serving_container_image_uri = serving_container_image_uri,
        )
    

### Compile and run the pipeline

In [13]:
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='ml_winequality.json')

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


The pipeline compilation generates the **ml_winequality.json** job spec file.

In [ ]:
### Create a run 

In [4]:
REGION="us-central1"

In [14]:
!$REGION

/bin/bash: us-central1: command not found


In [15]:
start_pipeline = pipeline_jobs.PipelineJob(
    display_name="winequality-pipeline-new",
    template_path="ml_winequality.json",
    enable_caching=False,
    location=REGION,
)

In [16]:
start_pipeline.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/829340160841/locations/us-central1/pipelineJobs/pipeline-winequality-new-20230118202215
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/829340160841/locations/us-central1/pipelineJobs/pipeline-winequality-new-20230118202215')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/pipeline-winequality-new-20230118202215?project=829340160841
PipelineJob projects/829340160841/locations/us-central1/pipelineJobs/pipeline-winequality-new-20230118202215 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/829340160841/locations/us-central1/pipelineJobs/pipeline-winequality-new-20230118202215 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/829340160841/locations/us-central1/pipelineJobs/pipeline-winequality-new-20230118202215 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob proje

### List all models 

In [17]:
DISPLAY_NAME = "winequality-new"
! gcloud ai models list --region={REGION} --filter={DISPLAY_NAME}

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
MODEL_ID             DISPLAY_NAME
8950589799073316864  winequality-new


### Schedule pipeline 

The scheduled jobs are supported by the Cloud Scheduler and Cloud Functions. 
Check that the APIs Cloud Scheduler, Cloud Functions are enabled. 

### Run recurrent pipeline

In [ ]:
from kfp.v2.google.client import AIPlatformClient

api_client = AIPlatformClient(
                project_id=PROJECT_ID,
                region=REGION,
                )

SERVICE_ACCOUNT = (
    "829340160841-compute@developer.gserviceaccount.com" 
)
response = api_client.create_schedule_from_job_spec(
    enable_caching=True,
    job_spec_path="ml_winequality.json",
    schedule="0 0 * * 1", 
    time_zone="Europe/Brussels",  # change this as necessary
    parameter_values={"display_name": DISPLAY_NAME},
    pipeline_root=PIPELINE_ROOT,  # this argument is necessary if you did not specify PIPELINE_ROOT as part of the pipeline definition.
    service_account=SERVICE_ACCOUNT,
    
)


/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/google/client/client.py:173: FutureWarning: AIPlatformClient will be deprecated in v2.0.0. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,


HttpError: <HttpError 403 when requesting https://serviceusage.googleapis.com/v1/projects/spmena-sandbox-in-apac-dv/services/cloudfunctions.googleapis.com:enable?alt=json returned "Permission denied to enable service [cloudfunctions.googleapis.com]
Help Token: AXyI5c9k_H2YC6-9codtK9iFGYU1tsIJMfm1Hpewyzq7iewVTXXg8kvzqosij7f24McFGQ2kWMh6nJyqv_JFNF2-ue6sgrvGCj7gBMSqhFBVuYY6". Details: "[{'@type': 'type.googleapis.com/google.rpc.PreconditionFailure', 'violations': [{'type': 'googleapis.com', 'subject': '?error_code=110002&service=serviceusage.googleapis.com&permission=serviceusage.services.enable&resource=spmena-sandbox-in-apac-dv'}]}, {'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'AUTH_PERMISSION_DENIED', 'domain': 'serviceusage.googleapis.com', 'metadata': {'service': 'serviceusage.googleapis.com', 'permission': 'serviceusage.services.enable', 'resource': 'spmena-sandbox-in-apac-dv'}}]">

Once the scheduled job is created, you can see it listed in the Cloud Scheduler panel in the Console.

### Test the batch prediction  

In [9]:
# Define variables 
job_display_name = "winequality-batch-prediction-job"
MODEL_NAME="winequality"
ENDPOINT_NAME="winequality_endpoint"
BUCKET_URI="gs://ae2-in-gcs-dv-assortment/pipeline_root_wine/829340160841/pipeline-winequality-20230116060532/get-wine-data_7095304939567579136"
#input_file_name="test.csv"
bigquery_source="bq://spmena-sandbox-in-apac-dv.model_deployment_monitoring_4263030881259618304.test"
bigquery_destination_prefix="bq://spmena-sandbox-in-apac-dv.model_deployment_monitoring_4263030881259618304.output"
sync: bool = True

# Get model id
MODEL_ID=!(gcloud ai models list --region=$REGION \
           --filter=display_name=$MODEL_NAME)
MODEL_ID=MODEL_ID[2].split(" ")[0]

model_resource_name = f'projects/{PROJECT_ID}/locations/{REGION}/models/{MODEL_ID}'
#gcs_source= [f"{BUCKET_URI}/{input_file_name}"]
bigquery_destination_prefix="bq://spmena-sandbox-in-apac-dv.model_deployment_monitoring_4263030881259618304.output"


def batch_prediction_job(
    project: str,
    location: str,
    model_resource_name: str,
    job_display_name: str,
    #gcs_source: str,
    bigquery_source: str,
    bigquery_destination_prefix: str,
    machine_type: str,
    #starting_replica_count: int = 1, # The number of nodes for this batch prediction job. 
    #max_replica_count: int = 1,
    #sync=sync
):   
    aiplatform.init(project=project, location=location)

    model = aiplatform.Model(model_resource_name)

    batch_prediction_job = model.batch_predict(
        job_display_name=job_display_name,
        instances_format='bigquery', #json
        #gcs_source=[f"{BUCKET_URI}/{input_file_name}"],
        bigquery_source="bq://spmena-sandbox-in-apac-dv.model_deployment_monitoring_4263030881259618304.test",
        bigquery_destination_prefix="bq://spmena-sandbox-in-apac-dv.model_deployment_monitoring_4263030881259618304.output",
        machine_type=machine_type, # must be present,
        #sync=sync
    )
    batch_prediction_job.wait()
    print(batch_prediction_job.display_name)
    print(batch_prediction_job.state)
    return batch_prediction_job

batch_prediction_job(PROJECT_ID, REGION, model_resource_name, job_display_name, bigquery_source, bigquery_destination_prefix, machine_type="n1-standard-2")

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/829340160841/locations/us-central1/batchPredictionJobs/469030907127267328
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/829340160841/locations/us-central1/batchPredictionJobs/469030907127267328')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/469030907127267328?project=829340160841
BatchPredictionJob projects/829340160841/locations/us-central1/batchPredictionJobs/469030907127267328 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/829340160841/locations/us-central1/batchPredictionJobs/469030907127267328 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/829340160841/locations/us-central1/batchPredictionJobs/469030907127267328 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/829340160841/locations/us-central1/batchPredictionJobs/469030

resource name: projects/829340160841/locations/us-central1/batchPredictionJobs/469030907127267328

### Send an online prediction request
Each prediction request must be max. 1.5 MB.

In [ ]:
ENDPOINT_NAME="winequality_endpoint"
instance = [[1,2,3,2,1,2,3,6,7,10]]
ENDPOINT_ID = !(gcloud ai endpoints list --region=$REGION \
              --format='value(ENDPOINT_ID)'\
              --filter=display_name=$ENDPOINT_NAME \
              --sort-by=creationTimeStamp | tail -1)
ENDPOINT_ID = ENDPOINT_ID[1]

def endpoint_predict(
    project: str, location: str, instances: list, endpoint: str
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint)

    prediction = endpoint.predict(instances=instances)
    return prediction

endpoint_predict(PROJECT_ID, REGION, instance, ENDPOINT_ID)


Prediction(predictions=[0.0], deployed_model_id='5026685687215161344', explanations=None)